# Fit the model

In [ ]:
# mr_hydra_multi_stock_training.py

import os
import gc
import glob
import numpy as np
import pandas as pd
import joblib
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score
from sklearn.model_selection import TimeSeriesSplit
from aeon.transformations.collection.convolution_based import MultiRocket, HydraTransformer
from aeon.utils.validation import check_n_jobs

# ----------------------
# Data utils
# ----------------------
def load_and_preprocess(filename):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"No file at {filename}")
    df = pd.read_csv(filename, parse_dates=["Date"])
    df["Direction"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df

def rolling_windows(df, window_size=20, drop_cols=None):
    if drop_cols is None:
        drop_cols = [
            "Date", "Close", "Target", "Direction",
            "High", "Low", "Open", "High_lag1", "Low_lag1", "Open_lag1"
        ]
    Xs, ys = [], []
    features = df.drop(columns=drop_cols, errors='ignore')
    y = df["Direction"].values
    for i in range(window_size, len(df)):
        Xs.append(features.iloc[i - window_size: i].values)
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

def reshape_for_aeon(X_3d):
    # sklearn-style (n_cases, time, features) -> aeon (n_cases, channels, time)
    return np.transpose(X_3d, (0, 2, 1))

# ----------------------
# Search (max 10 iterations)
# ----------------------
def _default_search_space(nf):
    """
    Returns up to 10 configs spanning windows + MR/Hydra budgets.
    Each item: (window, mr_kernels, hydra_groups, hydra_kernels, hydra_max_channels)
    """
    m32 = min(nf, 32)
    m64 = min(nf, 64)
    return [
        # Hydra Medium runs
        (20, 2048, 64, 8, m64),
        (20, 4096, 64, 8, m64),

        (64, 2048, 64, 8, m64),
        (64, 4096, 64, 8, m64),

        # (96, 2048, 64, 8, m64),
        # (96, 4096, 64, 8, m64),


    ]

def _cv_score_config(X, y, cfg, n_splits=3, gap=5, random_state=42):
    """
    Fit/score one config via TimeSeriesSplit CV.
    Leak-safe: fit transformers and classifier on each fold's train only.
    Returns (mean_bal_acc, std_bal_acc).
    """
    w, mrk, hg, hk, hch = cfg
    # TSCV on the windowed dataset
    tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)
    scores = []

    for tr_idx, te_idx in tscv.split(X):
        # Fresh transformers per fold
        hydra = HydraTransformer(
            n_groups=hg,
            n_kernels=hk,
            max_num_channels=hch,
            output_type="numpy",
            n_jobs=1,
            random_state=random_state,
        )
        mr = MultiRocket(
            n_kernels=mrk,
            max_dilations_per_kernel=32,
            n_features_per_kernel=4,
            normalise=True,       # avoid external scaling/leakage
            n_jobs=1,
            random_state=random_state,
        )

        Xtr, Xte = X[tr_idx], X[te_idx]
        ytr, yte = y[tr_idx], y[te_idx]

        # Fit/transform per fold
        X_hydra_tr = hydra.fit_transform(Xtr, ytr)
        X_hydra_te = hydra.transform(Xte)

        X_mr_tr = mr.fit_transform(Xtr, ytr)
        X_mr_te = mr.transform(Xte)

        Xtr_full = np.hstack([X_hydra_tr, X_mr_tr])
        Xte_full = np.hstack([X_hydra_te, X_mr_te])

        clf = RidgeClassifier(alpha=1.0)
        clf.fit(Xtr_full, ytr)
        y_pred = clf.predict(Xte_full)
        scores.append(balanced_accuracy_score(yte, y_pred))

        # Free fold memory
        del hydra, mr, X_hydra_tr, X_hydra_te, X_mr_tr, X_mr_te, Xtr_full, Xte_full, clf
        gc.collect()

    return float(np.mean(scores)), float(np.std(scores))

def mr_hydra_train_eval(
    df,
    stock_name,
    max_searches=10,
    n_splits_cv=3,
    gap=5,
    test_ratio=0.2,
    n_jobs=None,
    random_state=42,
    save_path="models/",
):
    """
    Runs a capped (<=10) MR+Hydra search across window sizes and kernel budgets,
    selects the best by CV balanced accuracy, refits on train split, evaluates on test,
    and saves the best model.
    """
    print(f"\n{'='*60}")
    print(f"Training model for: {stock_name}")
    print(f"{'='*60}")

    try:
        # Quick sample to get nf (feature count)
        Xw_sample, _ = rolling_windows(df, window_size=48)
        if len(Xw_sample) == 0:
            print("No data after windowing. Skipping.")
            return False
        _, _, nf = Xw_sample.shape

        # Build search space (<=10 configs)
        search_space = _default_search_space(nf)[:max_searches]

        results = []
        best_score = -np.inf
        best_cfg = None
        best_w_data = None  # cache (X,y) for the winning window

        if n_jobs is None:
            n_jobs = check_n_jobs(-1)

        # Evaluate each config
        for idx, cfg in enumerate(search_space, 1):
            w, mrk, hg, hk, hch = cfg
            # Build windows for this w
            Xw, yw = rolling_windows(df, window_size=w)
            if len(Xw) < 100:
                print(f"⚠️  [cfg {idx}/{len(search_space)}] window={w}: Only {len(Xw)} samples. Skipping.")
                continue

            X = reshape_for_aeon(Xw.astype("float32"))  # (n_cases, n_channels, n_timepoints)
            mean_ba, std_ba = _cv_score_config(X, yw, cfg, n_splits=n_splits_cv, gap=gap, random_state=random_state)

            results.append({
                "config_idx": idx,
                "window": w,
                "mr_kernels": mrk,
                "hydra_groups": hg,
                "hydra_kernels": hk,
                "hydra_max_channels": hch,
                "cv_bal_acc_mean": mean_ba,
                "cv_bal_acc_std": std_ba
            })
            print(f"[{idx}/{len(search_space)}] w={w}, MR={mrk}, Hydra=({hg},{hk},{hch}) "
                  f"-> CV bal_acc {mean_ba:.4f} ± {std_ba:.4f}")

            # Track best
            if mean_ba > best_score:
                best_score = mean_ba
                best_cfg = cfg
                best_w_data = (X, yw)

            # Free per-config memory (except best cached data)
            if best_w_data is None or best_w_data[0] is not X:
                del X, yw
            del Xw
            gc.collect()

        if best_cfg is None:
            print("No valid configuration found. Skipping.")
            return False

        print(f"\nBest config by CV: window={best_cfg[0]}, MR={best_cfg[1]}, "
              f"Hydra=(groups={best_cfg[2]}, kernels={best_cfg[3]}, max_ch={best_cfg[4]}) "
              f"with CV bal_acc={best_score:.4f}")

        # Final train/test split on the best windowed data (chronological split)
        X, y = best_w_data
        ns = X.shape[0]
        split_idx = int((1 - test_ratio) * ns)
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]

        # Refit transformers on full train, transform train & test
        w, mrk, hg, hk, hch = best_cfg

        hydra = HydraTransformer(
            n_groups=hg,
            n_kernels=hk,
            max_num_channels=hch,
            output_type="numpy",
            n_jobs=1,
            random_state=random_state,
        )
        mr = MultiRocket(
            n_kernels=mrk,
            max_dilations_per_kernel=32,
            n_features_per_kernel=4,
            normalise=True,
            n_jobs=1,
            random_state=random_state,
        )

        print("\nFitting best transformers on training split...")
        X_hydra_train = hydra.fit_transform(X_train, y_train)
        X_hydra_test = hydra.transform(X_test)

        X_mr_train = mr.fit_transform(X_train, y_train)
        X_mr_test = mr.transform(X_test)

        X_train_full = np.hstack([X_hydra_train, X_mr_train])
        X_test_full = np.hstack([X_hydra_test, X_mr_test])

        # Train classifier (fixed alpha to keep total iterations <= 10)
        clf = RidgeClassifier(alpha=1.0)
        clf.fit(X_train_full, y_train)

        y_pred = clf.predict(X_test_full)
        acc = accuracy_score(y_test, y_pred)
        print(f"\nTest accuracy: {acc:.4f}")
        print(classification_report(y_test, y_pred, target_names=["Down", "Up"]))

        # Save model components
        os.makedirs(save_path, exist_ok=True)
        model_data = {
            'hydra': hydra,
            'multirocket': mr,
            'classifier': clf,
            'best_config': {
                'window_size': w,
                'mr_kernels': mrk,
                'hydra_groups': hg,
                'hydra_kernels': hk,
                'hydra_max_channels': hch,
            },
            'n_features': X.shape[1],
            'stock_name': stock_name,
            'test_accuracy': acc,
            'cv_results': pd.DataFrame(results).sort_values("cv_bal_acc_mean", ascending=False).to_dict(orient='list'),
            'drop_cols': [
                "Date", "Close", "Target", "Direction",
                "High", "Low", "Open", "High_lag1", "Low_lag1", "Open_lag1"
            ],
        }
        model_filename = f"{save_path}/{stock_name}_mr_hydra_best.pkl"
        joblib.dump(model_data, model_filename)
        print(f"✓ Best model saved to {model_filename}")

        # Free memory
        del X, y, X_train, X_test, X_hydra_train, X_hydra_test, X_mr_train, X_mr_test, X_train_full, X_test_full
        del hydra, mr, clf
        gc.collect()

        return True

    except Exception as e:
        print(f"❌ Error training {stock_name}: {str(e)}")
        return False

# ----------------------
# Batch over all stocks
# ----------------------
def train_all_stocks(
    data_dir="DATA/",
    save_path="models-grid-2/",
    max_searches=10,
    **kwargs
):
    csv_files = glob.glob(os.path.join(data_dir, "*.csv"))
    if not csv_files:
        print(f"No CSV files found in {data_dir}")
        return

    print(f"Found {len(csv_files)} datasets to process")
    results = []
    successful = failed = 0

    for i, filepath in enumerate(csv_files, 1):
        filename = os.path.basename(filepath)
        stock_name = filename.replace('_daily_features.csv', '').replace('.csv', '')

        print(f"\n[{i}/{len(csv_files)}] Processing {stock_name}...")
        try:
            df = load_and_preprocess(filepath)
            print(f"Loaded {len(df)} rows for {stock_name}")

            ok = mr_hydra_train_eval(
                df,
                stock_name,
                save_path=save_path,
                max_searches=max_searches,
                **kwargs
            )
            if ok:
                successful += 1
                results.append({'stock': stock_name, 'status': 'success', 'rows': len(df)})
            else:
                failed += 1
                results.append({'stock': stock_name, 'status': 'failed', 'rows': len(df)})

            # Free per-stock DataFrame memory
            del df
            gc.collect()

        except Exception as e:
            print(f"❌ Failed to process {stock_name}: {str(e)}")
            failed += 1
            results.append({'stock': stock_name, 'status': 'error', 'error': str(e)})

    print(f"\n{'='*60}")
    print("TRAINING SUMMARY")
    print(f"{'='*60}")
    print(f"Total datasets: {len(csv_files)}")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")

    results_df = pd.DataFrame(results)
    os.makedirs(save_path, exist_ok=True)
    summary_path = f"{save_path}/training_summary.csv"
    results_df.to_csv(summary_path, index=False)
    print(f"\nDetailed results saved to: {summary_path}")

    successful_models = [r['stock'] for r in results if r['status'] == 'success']
    if successful_models:
        print(f"\nSuccessfully trained models for:")
        for stock in successful_models:
            print(f"  - {stock}")

if __name__ == "__main__":
    train_all_stocks(
        data_dir="data/",
        save_path="models-grid-2/",
        # you can tweak below without increasing iteration count:
        n_splits_cv=3,
        gap=5,
        test_ratio=0.2,
        max_searches=4,
        random_state=42
    )


Found 5 datasets to process

[1/5] Processing AAPL...
Loaded 11224 rows for AAPL

Training model for: AAPL


c:\Users\gee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Users\gee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Users\gee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


[1/4] w=20, MR=2048, Hydra=(64,8,17) -> CV bal_acc 0.5052 ± 0.0164


c:\Users\gee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Users\gee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Users\gee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


[2/4] w=20, MR=4096, Hydra=(64,8,17) -> CV bal_acc 0.5052 ± 0.0164


...

[3/4] w=64, MR=2048, Hydra=(64,8,17) -> CV bal_acc 0.5044 ± 0.0102


c:\Users\gee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Users\gee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Users\gee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


[4/4] w=64, MR=4096, Hydra=(64,8,17) -> CV bal_acc 0.5044 ± 0.0102

Best config by CV: window=64, MR=2048, Hydra=(groups=64, kernels=8, max_ch=17) with CV bal_acc=0.5044

Fitting best transformers on training split...


c:\Users\gee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(



Test accuracy: 0.5104
              precision    recall  f1-score   support

        Down       0.46      0.39      0.42       895
          Up       0.55      0.61      0.58      1072

    accuracy                           0.51      1967
   macro avg       0.50      0.50      0.50      1967
weighted avg       0.50      0.51      0.51      1967

✓ Best model saved to models-grid-2//MSFT_mr_hydra_best.pkl

TRAINING SUMMARY
Total datasets: 5
Successful: 5
Failed: 0

Detailed results saved to: models-grid-2//training_summary.csv

Successfully trained models for:
  - AAPL
  - GE
  - IBM
  - JNJ
  - MSFT
